<a href="https://colab.research.google.com/github/SolinSM/TDCGAN/blob/main/TDCGAN_try_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **For Dataset**

## **Compare sentence Column - Dataset**

*   Used with our-dataset security course




### **Compare 'sentence' column in 2 excel file**

In [ ]:
import pandas as pd
import numpy as np

"""  ##### In 2 excel Files ##### """
file_1 = '/content/SQLiDataset.xlsx'
file_2 = '/content/c_our_Dataset-old.xlsx'

df1 = pd.read_excel(file_1)
df2 = pd.read_excel(file_2)

dff1 = pd.DataFrame()
dff1['sentence_1'] = df1['sentence']
dff1['sentence_2'] = df2['sentence']

# use equal to compare when index not same in files
#dff1['diff'] = np.where(df1['sentence'] == df2['sentence'], 0,1)
dff1['diff'] = np.where( df1['sentence'].equals(df2['sentence']) , 0,1)

dff1.to_excel('/content/new_SQLiDataset.xlsx', index=False)


### **Compare 'sentence' in 2 column in same excel file**

In [ ]:
import pandas as pd
import numpy as np

file_2 = '/content/new_SQLiDataset.xlsx'

df2 = pd.read_excel(file_2)

dff1 = pd.DataFrame()

dff1['sentence_1'] = df2['sentence_1']
dff1['sentence_2'] = df2['sentence_2']
dff1['diff'] = np.where(df2['sentence_1'] == df2['sentence_2'], 0,1)

dff1.to_excel('/content/new_SQLiDataset_1.xlsx', index=False)


## **Read Specific Column (Sentence) + add new Column "len_payload" + values**

In [28]:
# Import pandas library
import pandas as pd

# Step 1: Define CSV File Path
file_path =  '/content/s_SQLiDataset.csv'
#file_path =  '/content/s_SQLiDataset.xlsx'

# Step 2: Read the CSV file into a DataFrame
df = pd.read_csv(file_path, encoding = "ISO-8859-1" , usecols=['sentence'])
#df = pd.read_excel(file_path)

total_rows_befor = len(df)

# ## show head 10 of data before adding new column
# print('----------------------')
# print('Before adding new column')
# print(df.head())
# print('----------------------')

# Step 3: Add the new "len_payload" column to the DataFrame and dataType it as int
df['len_payload'] = 0
df = df.astype({'len_payload': int})

# Step 4: Set length payload to a new added column "len_payload"
# iterating over rows using iterrows() function
for i, j in df.iterrows():

    # Step 4.1: Get 'sentence' value
    sentence = df['sentence'][i]

    # Step 4.2: Drop row with null payload
    if pd.isna(sentence):
      print('index sentence with value nan : ', i)
      df.drop( labels = i, axis=0, inplace = True )

    else:
      #print('index : ', i)
      # Step 4.3: Get the payload value and strip White Spaces
      sentence = str(sentence).strip()

      # Step 4.4: Find the sentence length
      len_payload = len(sentence)

      # Step 4.5: Update value of "len_payload" payload/sentence length
      df.loc[i, 'len_payload'] = len_payload


# Step 5: Write the DataFrame back to the CSV file
df.to_csv(file_path, index=False)
#df.to_excel('/content/new_len_SQLiDataset_1.xlsx', index=False)

# Check length file
print('----------------------')
total_rows_after = len(df)
print('Total rows total_rows_befor: ', total_rows_befor)
print('Total rows total_rows_after: ', total_rows_after)


## Check new data is added
print('----------------------')
print('Aefore adding new column')
pd.read_csv(file_path, encoding = "ISO-8859-1").head()
#pd.read_excel(file_path)


index nan :  12211
index nan :  12212
index nan :  12213
----------------------
Total rows total_rows_befor:  39688
Total rows total_rows_after:  39685
----------------------
Aefore adding new column


,sentence,len_payload
0,-3752,5
1,02:00 AM,8
2,0,1
3,21%,3
4,26%,3


# **Decode Payload**

In [7]:
import base64
import urllib.parse

### ------------- Decode function ------------- ###
def unicode_decode(str):
  return str.encode('utf-8').decode('unicode-escape')

def url_decode(str):
  return urllib.parse.unquote(str)

def base64_decode(base64_string):
  base64_bytes = base64_string.encode("ascii")
  sample_string_bytes = base64.b64decode(base64_bytes)
  sample_str = sample_string_bytes.decode("ascii")
  return sample_str

def unicode_url_decoding(sql_payload):
  if '%20' in sql_payload or '%' in sql_payload:
    try:
      d_payload = url_decode(sql_payload)
    except: # Exception as e:
      #print(f"Decoding error in - unicode_url_decoding() - with: {e}")
      d_payload = sql_payload

  elif sql_payload.startswith(('\\u', '\\U')):
    try:
      d_payload = unicode_decode(sql_payload)
    except: # Exception as e:
      d_payload = sql_payload

  else:
    d_payload = sql_payload

  return d_payload



### ------------- Decode function ------------- ###
def decode_payload(sql_payload):

  if sql_payload[-1] == '=' and sql_payload[-2] == '=':
    try:
      base64_payload = base64_decode(sql_payload)
      if base64_payload:
        decode_payload = unicode_url_decoding(base64_payload)
      else:
        decode_payload = unicode_url_decoding(sql_payload)
    except Exception as e:
        decode_payload = unicode_url_decoding(sql_payload)

  else:
    sql_payload += '=='
    try:
      base64_payload = base64_decode(sql_payload)
      if base64_payload:
        decode_payload = unicode_url_decoding(base64_payload)
      else:
        decode_payload = unicode_url_decoding(sql_payload)
    except Exception as e:
      sql_payload = sql_payload[:-2]      # remove last 2 chat == added in this section
      decode_payload = unicode_url_decoding(sql_payload)

  return decode_payload


### ------------- main ------------- ###
# Example usage:
sql_payload = "SELECT%20*%20FROM%20users%20WHERE%20username%3D%27john%27%3B"    # url encoded
sql_payload = "U0VMRUNUICogRlJPTSB1c2VycyBXSEVSRSB1c2VybmFtZT0nam9obic7=="      # base64 encoded
sql_payload = "\\u0053\\u0045\\u004C\\u0045\\u0043\\u0054\\u0020\\u002A\\u0020\\u0046\\u0052\\u004F\\u004D\\u0020\\u0075\\u0073\\u0065\\u0072\\u0073\\u0020\\u0057\\u0048\\u0045\\u0052\\u0045\\u0020\\u0075\\u0073\\u0065\\u0072\\u006E\\u0061\\u006D\\u0065\\u003D\\u0027\\u006A\\u006F\\u0068\\u006E\\u0027\\u003B"   # unicode encoded
sql_payload = "U0VMRUNUJTIwKiUyMEZST00lMjB1c2VycyUyMFdIRVJFJTIwdXNlcm5hbWUlM0QlMjdqb2huJTI3JTNC"   # base64 + url encoded
sql_payload = "XHUwMDUzXHUwMDQ1XHUwMDRDXHUwMDQ1XHUwMDQzXHUwMDU0XHUwMDIwXHUwMDJBXHUwMDIwXHUwMDQ2XHUwMDUyXHUwMDRGXHUwMDREXHUwMDIwXHUwMDc1XHUwMDczXHUwMDY1XHUwMDcyXHUwMDczXHUwMDIwXHUwMDU3XHUwMDQ4XHUwMDQ1XHUwMDUyXHUwMDQ1XHUwMDIwXHUwMDc1XHUwMDczXHUwMDY1XHUwMDcyXHUwMDZFXHUwMDYxXHUwMDZEXHUwMDY1XHUwMDNEXHUwMDI3XHUwMDZBXHUwMDZGXHUwMDY4XHUwMDZFXHUwMDI3XHUwMDNC"  # base64 + unicode encoded
print("Befor Decoded payload:", sql_payload)

decoded_payload = decode_payload(sql_payload)
if decoded_payload:
    print("After Decoded payload:", decoded_payload)
else:
    print("Unable to decode payload using any method.")

Befor Decoded payload: XHUwMDUzXHUwMDQ1XHUwMDRDXHUwMDQ1XHUwMDQzXHUwMDU0XHUwMDIwXHUwMDJBXHUwMDIwXHUwMDQ2XHUwMDUyXHUwMDRGXHUwMDREXHUwMDIwXHUwMDc1XHUwMDczXHUwMDY1XHUwMDcyXHUwMDczXHUwMDIwXHUwMDU3XHUwMDQ4XHUwMDQ1XHUwMDUyXHUwMDQ1XHUwMDIwXHUwMDc1XHUwMDczXHUwMDY1XHUwMDcyXHUwMDZFXHUwMDYxXHUwMDZEXHUwMDY1XHUwMDNEXHUwMDI3XHUwMDZBXHUwMDZGXHUwMDY4XHUwMDZFXHUwMDI3XHUwMDNC
After Decoded payload: SELECT * FROM users WHERE username='john';


# **Text Processing**
* Input text String
* Remove white spaces from start & end in text, using strip()
* Remove accents
* Convert all letters of the string to lower case
* Tokenization
* Remove stop words, and except some important words used in queries

ref code: [https://www.geeksforgeeks.org/python-ai/?ref=shm](https://www.geeksforgeeks.org/python-ai/?ref=shm)

## **Stop words + Tokenization**

In [10]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
#import unicodedata
#import string

#from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer      ## WordPunctTokenizer --> splits words based on punctuation boundaries.



"""
Filters stop words from a list of tokens, keeping words in the exception list.

Args:
    word_tokens: A list of tokens (words).
    except_stop_word: A set of stop words to be excluded from removal (optional).

Returns:
    A list of tokens with stop words removed, except for those in the exception list.
"""
def filter_stop_words(word_tokens):
    stop_words = set(stopwords.words('english'))  # Load default stop words
    except_stop_word = set(['and', 'or', 'where', 'from', 'over'])  # Exception list defined here

    filtered_sentence = []
    for w in word_tokens:
        if w in except_stop_word or w not in stop_words:
            filtered_sentence.append(w)

    return filtered_sentence




text = "     SELECT * FROM users WHERE username='john' and    fname='john'; is"

# convert to lower case
text = text.lower()

# Remove spaces in start and end of sentence
text = text.strip()

# Remove accents function such as é
#text = ''.join(x for x in unicodedata.normalize('NFKD', text) if x in string.ascii_letters or x == " ")

#word_tokens = word_tokenize(text)
tokenizer   = WordPunctTokenizer()
word_tokens = tokenizer.tokenize(text)

# stop_words  (filter tokeniz word by remove any stop word with except some word as from, where ... etc)
filtered_tokens = filter_stop_words(word_tokens)

print(word_tokens)
print('------')
print(filtered_tokens)

['select', '*', 'from', 'users', 'where', 'username', "='", 'john', "'", 'and', 'fname', "='", 'john', "';", 'is']
------
['select', '*', 'from', 'users', 'where', 'username', "='", 'john', "'", 'and', 'fname', "='", 'john', "';"]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk.tokenize import WordPunctTokenizer
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from nltk.corpus import stopwords

# Define CSV file path (replace with your actual path)
csv_path = "your_data.csv"

# Stop words with exceptions
stop_words = set(stopwords.words('english'))
except_stop_word = ['and', 'or', 'where', 'from', 'over']

# Function for text pre-processing (optional)
"""
Preprocesses text data by performing various cleaning steps.

Args:
    text: The text string to be preprocessed.

Returns:
    The preprocessed text string.
"""
def preprocess_text(text):
  # Convert to lowercase
  text = text.lower()

  # Remove spaces in start and end
  text = text.strip()

  # Remove accents (optional)
  # text = ''.join(x for x in unicodedata.normalize('NFKD', text) if x in string.ascii_letters or x == " ")

  return text

# Read data from CSV using pandas
data = pd.read_csv(csv_path)

# Get sentences as a list
sentences = data["sentence"].apply(preprocess_text)  # Apply pre-processing

# Tokenization
tokenizer = WordPunctTokenizer()
tokenized_sentences = []
for sentence in sentences:
  word_tokens = tokenizer.tokenize(sentence)

  # Filter stop words (considering exceptions)
  filtered_sentence = []
  for w in word_tokens:
    if w not in except_stop_word and w not in stop_words:
      filtered_sentence.append(w)

  tokenized_sentences.append(filtered_sentence)

# Create vocabulary
all_tokens = [token for sentence in tokenized_sentences for token in sentence]
vocabulary = set(all_tokens)

# One-hot encoding (using scikit-learn's OneHotEncoder)
encoder = OneHotEncoder(sparse=False)  # Dense output for easier visualization
encoded_sentences = []
for tokens in tokenized_sentences:
  encoded_sentences.append(encoder.fit_transform(np.array([tokens]).reshape(1, -1))[0])

# Print results (optional)
for i, sentence in enumerate(sentences):
  print(f"Sentence {i+1}:", sentence)
  print("Tokens:", tokenized_sentences[i])
  print("One-hot encoded:", encoded_sentences[i])
  print("-" * 20)

# **Read Multi Files csv (done below)**

In [9]:
import os
import pandas as pd
import numpy as np
import random


""" ----------------- Read CSV File Function ----------------- """
def read_csv_files(dataset_directory, percent):
  files = [f for f in os.listdir(dataset_directory) if f.endswith('.csv')]

  if files == []:
    print('Not found any csv files')
  else:
    print('Your files are: ', files)

    np_array_values = []
    data_df = pd.DataFrame()
    firstFile = True

    for file in files:
      file_path = os.path.join(dataset_directory, file)   # csv_file_path
      print('File Path: ', file_path)

      try:
        df = ''
        df = pd.read_csv(file_path, encoding = "ISO-8859-1")  #.head()   #,low_memory=False   ISO-8859-1
        total_rows = len(df)
        print('Total rows in df/file: ', total_rows)

        num_rows = int(total_rows * (percent / 100))
        print('Total rows in df/file 100%: ', num_rows)

        """ Start From Net Code """
        ##np_array_values.append(df)
        #print('np_array_values: ' , np_array_values)

        ###merge_values = np.vstack(np_array_values)
        #print('merge_values: ', merge_values)

        #new_data = pd.DataFrame(merge_values)
        #new_data = pd.DataFrame(merge_values , columns=['sentence','attack_type_code', 'attack_type', 'http_method', 'param_as', 'flag_success_attack', 'http_status', 'param_length'])
        ###new_data = pd.DataFrame(merge_values , columns=['sentence','attack_type_code', 'param_length'])
        ###print('new_data: ' , new_data)
        """ End From Net Code """

        """ Start From Teacher Code """
        # Generate a list of random indices
        random_indices = random.sample(range(total_rows), num_rows)
        #print('random_indices: ' , random_indices)

        # Select the random rows from the DataFrame
        temp_df = df.iloc[random_indices]
        if(firstFile):
          # Concatenate all DataFrames into one
          data_df = temp_df.copy()
          firstFile = False
        else:
          # Concatenate all DataFrames into one
          data_df = pd.concat([data_df,temp_df], ignore_index=True)

        print(data_df)
        return data_df
        """ End From Teacher Code """

        ## Add DataFrame to new CSV file
        #new_csv_file_path = os.path.join(dataset_directory, 'new_sqli.csv')  # "/content/dataset/new_sqli.csv"
        #df.to_csv(new_csv_file_path, index=False)

      except Exception as e:
        print('Can not Read File called : ', file)
        print('File path: ', file_path)
        print("Errpr Exception e : ", e)



""" ----------------- Apply Code ----------------- """
dataset_directory = "/content/datasets"   #files_path
percent = 100
data = read_csv_files(dataset_directory,percent)


Your files are:  ['SQLiDataset_2.csv']
File Path:  /content/datasets/SQLiDataset_2.csv
Total rows in df/file:  39685
Total rows in df/file 100%:  39685
                                                sentence  attack_type  \
28661                                do cotaredo, 147 9e            0   
38839                   vidom.kapatos@discotecateatre.om            0   
6589    select * from users where id = 1 or " )  ( " ...            1   
17838  1%' or 4915 =  ( select count ( * )  from doma...            1   
18982                1'+ ( select idnb where 2338 = 2338            1   
...                                                  ...          ...   
12402  1'  )  )   )  and 6537 = dbms_pipe.receive_mes...            1   
1083   '  )  )   )  UNION ALL SELECT NULL, NULL, NULL...            1   
31117                                          incr622to            0   
34798                                        pradoluengo            0   
10326  " )  AND 7767 = CAST ( CHR ( 58 ) ||CH

# **Read random n row**

In [10]:
""" ----------------- Read Random n rows Function ----------------- """
def read_random_n_rows(df, n, percent):
    # Read the entire CSV file into a DataFrame
    #df = read_csv_files(path,percent)

    # Check if requested sample size is greater than the total number of rows
	# n --> requested sample size
	# len(df) --> total number of rows
    if n >= len(df):
        return df

    # Randomly select n rows from the DataFrame
    random_sample = df.sample(n=n, random_state=42)
    random_sample.reset_index(drop=True,inplace=True)
    return random_sample


""" ----------------- Apply Read Random n rows Function ----------------- """
nrows = 12000
random_sample = read_random_n_rows(data, nrows, percent)
random_sample

,sentence,attack_type,len_payload
0,"UNION ALL SELECT 1,2,3,4,5,6#",1,29
1,The man &apos;s distraught nine-year-old son ...,0,128
2,html M Series United States Deluxe Laminated 5...,0,62
3,It attractive able determine daily working ro...,0,50
4,' AND 9263 = ( SELECT 9263 FROM PG_SLEEP ( 5 ...,1,51
...,...,...,...
11995,"c/ veinat de cerdanyola, 3 6e",0,29
11996,naredo reggiardo,0,16
11997,"' ) UNION ALL SELECT NULL, NULL, NULL, NULL, ...",1,220
11998,hornillos del camino,0,20


# **Up Classes (not work)**

In [8]:
"""  No Neeeed (Long Time + 23 min & not end loop)   """

""" ----------------- Read Up Class Function ----------------- """

def upclasses(df,label,num):
  #calculate the occurrences of each class and identify classes with fewer
  #instances than the specified minimum (min_instances_per_class).
  #We then duplicate rows for those classes using random sampling.
  # Set the minimum number of instances per class
  min_instances_per_class = num

  # Count the occurrences of each class in the original labels
  class_counts = df[label].value_counts()
  #print('Count the occurrences of each class in the original labels: ' , class_counts)        # list of sentence and at end --> Name: count, Length: 38025, dtype: int64

  # Identify classes with fewer instances than the minimum
  classes_to_duplicate = class_counts[class_counts < min_instances_per_class].index
  #print('classes with fewer instances than the minimum (classes_to_duplicate): ' , classes_to_duplicate)

  # Create an empty DataFrame to store duplicated rows
  df_duplicated = ''
  df_duplicated = pd.DataFrame(columns=df.columns)

  # Duplicate rows for classes with fewer instances than the minimum
  # df[label] : list of all values in column label that selected
  # class_label : from classes_to_duplicate
  # class_df : if equal then get all values in row index (all column not only selected label)
  for class_label in classes_to_duplicate:
    class_df = df[df[label] == class_label]

    # Duplicate rows using random sampling with replacement (duplicate class_df in num rows where num is param)
    duplicated_rows = class_df.sample(n=min_instances_per_class, replace=True, random_state=42)

    # Concatenate the duplicated rows to the new DataFrame
    df_duplicated = pd.concat([df_duplicated, duplicated_rows])

  print('===================')
  print('df_duplicated: ', df_duplicated)

  # Concatenate the original DataFrame with the duplicated rows
  df_balanced = pd.concat([df, df_duplicated], ignore_index=True)
  return df_balanced



""" ----------------- Apply Up Class Function ----------------- """

upclasses(data,'sentence',2000)

KeyboardInterrupt: 

# **Word Embeddings in NLP (One-hot encoding)**

It is a way to represent words as a numeric representations in a lower-dimensional space and to preserve syntactical and semantic information. It can be used as a method to extracting features out of text, and use them as input into ML model to work with text data.

Here, we will use one of a traditional Approachs which is One-hot encoding. It is a simple method used to represent words in NLP. Each word in the vocabulary is represented as a unique vector, where the vector's dimensional is equal to the vocabulary size. This encoding works by set 0 to all elements in the vector, except the element corresponding to the index of the word in the vocabulary, which is set to 1.

لماذا هي وليس غيرها
تعتمد الأساليب مثل
 Bag of Words (BOW) وCountVectorizer وTFIDF
 على عدد الكلمات في الجملة ولكنها لا تحفظ أي معلومات نحوية أو دلالية. في هذه الخوارزميات، حجم المتجه هو عدد العناصر الموجودة في المفردات. يمكننا الحصول على مصفوفة متفرقة إذا كانت معظم العناصر صفرًا. ستعني ناقلات المدخلات الكبيرة عددًا كبيرًا من الأوزان مما سيؤدي إلى عمليات حسابية عالية مطلوبة للتدريب.
 توفر Word Embeddings حلاً لهذه المشكلات.


 [reff](https://www.geeksforgeeks.org/word-embeddings-in-nlp/)



# **TfidfVectorizer Function**

In [46]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

data = {
	'text': [ 'This is the first doc.', 'this doc is the second doc this nice', 'and the third, the 4', 'solin sm', 'hi doc'],
	'cat':[ 'cat1', 'cat2', 'cat2', 'cat1', 'cat1' ]
}

df = pd.DataFrame(data)

label_encoder = LabelEncoder()
df['cat'] = label_encoder.fit_transform(df['cat'])

X = df['text']
y = df['cat']



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)	# ,stratify=y
#print(f"X_train shape: {X_train.shape}")
#print(f"y_train shape: {y_train.shape}")
#print(f"X_test shape : {X_test.shape}")
#print(f"y_test shape : {y_test.shape}")


tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_train_tfidf_selected_features = tfidf_vectorizer.get_feature_names_out()
#print("Shape of X_train_tfidf:", X_train_tfidf[:5].toarray().shape)
#print("Shape of X_train_tfidf:", len(tfidf_vectorizer.get_feature_names_out()) )

X_test_tfidf = tfidf_vectorizer.fit_transform(X_test)
X_test_tfidf_selected_features = tfidf_vectorizer.get_feature_names_out()
#print("Shape of X_test_tfidf:", X_test_tfidf[:5].toarray().shape)
#print("Shape of X_test_tfidf:", len(tfidf_vectorizer.get_feature_names_out()))


print('Shape of X_train_tfidf : ', X_train_tfidf.shape)
print('Shape of X_test_tfidf : ', X_test_tfidf.shape)

df_tfidf = pd.DataFrame(X_train_tfidf[:5].toarray(), index = X_train, columns=X_train_tfidf_selected_features)

print(df_tfidf)

Shape of X_train_tfidf :  (4, 10)
Shape of X_test_tfidf :  (1, 6)
                            and       doc     first        hi        is  \
text                                                                      
hi doc                  0.00000  0.619130  0.000000  0.785288  0.000000   
and the third, the 4    0.47212  0.000000  0.000000  0.000000  0.000000   
This is the first doc.  0.00000  0.382743  0.485461  0.000000  0.485461   
solin sm                0.00000  0.000000  0.000000  0.000000  0.000000   

                              sm     solin       the    third      this  
text                                                                     
hi doc                  0.000000  0.000000  0.000000  0.00000  0.000000  
and the third, the 4    0.000000  0.000000  0.744450  0.47212  0.000000  
This is the first doc.  0.000000  0.000000  0.382743  0.00000  0.485461  
solin sm                0.707107  0.707107  0.000000  0.00000  0.000000  


## **Class TDCGAN**

*   init definition
*   build_generator
*   build_discriminator
*   build_combined_model
*   evaluate_on_real
*   generate_synthetic_data    
*   plot_losses
*   plot_architecture
*   print_parameters_and_summary
*   train













# **TDCGAN Model**

## **libraries**

In [33]:
#for read csv file
import os
import pandas as pd
import numpy as np
import random


# pre-process
##for stop word
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

## not used now
#import unicodedata     # Remove accents
#import string

## for tokenization
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer      ## WordPunctTokenizer --> splits words based on punctuation boundaries.

## for divide data to (train / test/ validate)
from sklearn.model_selection import train_test_split


# for one-hor encode (sentence to 2D)
from sklearn.preprocessing import OneHotEncoder

# for TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

# for TDCGAN
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import (
    Input,
    Dense,
    Dropout,
    Embedding,
    Concatenate,
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## **definition**

In [34]:
GENERATOR_DROPOUT_RATE = 0.1
DISCRIMINATOR_DROPOUT_RATE = 0.3      #Adjust the dropout rate to prevent overfitting during training.
LEAKY_RELU_ALPA = 0.2

NUM_EPOCHS = 1000
BATCH_SIZE = 128
OPTIMIZER_LR = 0.0001                 # learning rate
OPTIMIZER_BETAS = (0.5, 0.999)


## **Color list + TextStyle**

In [35]:
colors_list = [
    'Red', 'Green', 'Blue', 'Purple', 'Orange', 'Pink', 'Brown', 'Yellow',
    'Cyan', 'Magenta', 'Lime', 'Teal', 'Lavender', 'Maroon', 'Navy', 'Olive', 'Silver', 'Gold',
    'Indigo', 'Turquoise', 'Beige', 'Crimson', 'Salmon','Tan','Lime', 'Fuchsia', 'Plum',
    'Tomato', 'Violet']

class TextStyle:
    # Font Styles
    BOLD = '\033[1m'
    DIM = '\033[2m'
    UNDERLINE = '\033[4m'
    BLINK = '\033[5m'
    REVERSE = '\033[7m'
    RESET_ALL = '\033[0m'

    # Font Colors
    BLACK = '\033[30m'
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    BLUE = '\033[34m'
    MAGENTA = '\033[35m'
    CYAN = '\033[36m'
    WHITE = '\033[37m'

    # Background Colors
    BG_BLACK = '\033[40m'
    BG_RED = '\033[41m'
    BG_GREEN = '\033[42m'
    BG_YELLOW = '\033[43m'
    BG_BLUE = '\033[44m'
    BG_MAGENTA = '\033[45m'
    BG_CYAN = '\033[46m'
    BG_WHITE = '\033[47m'


## **Read Multi Files csv**

In [36]:
""" ----------------- Read CSV File Function ----------------- """
def read_csv_files(dataset_directory, percent):
  print(f"{TextStyle.BOLD}{TextStyle.BLACK}------------ Handling Read CSV Files ------------{TextStyle.RESET_ALL}")

  files = [f for f in os.listdir(dataset_directory) if f.endswith('.csv')]

  if files == []:
    print('Not found any csv files')
  else:
    print('Your files are: ', files)

    np_array_values = []
    data_df = pd.DataFrame()
    firstFile = True

    for file in files:
      file_path = os.path.join(dataset_directory, file)   # csv_file_path
      print('File Path: ', file_path)

      try:
        df = ''
        df = pd.read_csv(file_path, encoding = "ISO-8859-1")  #.head()   #,low_memory=False   ISO-8859-1
        total_rows = len(df)
        print('Total rows in df/file: ', total_rows)

        num_rows = int(total_rows * (percent / 100))
        print('Total rows in df/file 100%: ', num_rows)


        """ Start From Teacher Code """
        # Generate a list of random indices
        random_indices = random.sample(range(total_rows), num_rows)
        #print('random_indices: ' , random_indices)

        # Select the random rows from the DataFrame
        temp_df = df.iloc[random_indices]
        if(firstFile):
          # Concatenate all DataFrames into one
          data_df = temp_df.copy()
          firstFile = False
        else:
          # Concatenate all DataFrames into one
          data_df = pd.concat([data_df,temp_df], ignore_index=True)

        print(data_df)
        return data_df
        """ End From Teacher Code """

        ## Add DataFrame to new CSV file
        #new_csv_file_path = os.path.join(dataset_directory, 'new_sqli.csv')  # "/content/dataset/new_sqli.csv"
        #df.to_csv(new_csv_file_path, index=False)

      except Exception as e:
        print('Can not Read File called : ', file)
        print('File path: ', file_path)
        print("Errpr Exception e : ", e)


#""" ----------------- Apply Code ----------------- """
#dataset_directory = "/content/datasets"   #files_path
#percent = 100
#data = read_csv_files(dataset_directory,percent)


## **pre-process data**

### **Functions to used here**

In [57]:
"""
Filters stop words from a list of tokens, keeping words in the exception list.
Args:
    word_tokens: A list of tokens (words).
    except_stop_word: A set of stop words to be excluded from removal (optional).
Returns:
    A list of tokens with stop words removed, except for those in the exception list.
"""
def filter_stop_words(word_tokens):
    stop_words = set(stopwords.words('english'))  # Load default stop words
    except_stop_word = set(['and', 'or', 'where', 'from', 'over'])  # Exception list defined here

    filtered_sentence = []
    for w in word_tokens:
        if w in except_stop_word or w not in stop_words:
            filtered_sentence.append(w)

    return filtered_sentence




# Tokenization + stop word
def tokenization_text(sentences):
    print(f"{TextStyle.BOLD}{TextStyle.BLACK}------------ Tokenization + stop word ------------{TextStyle.RESET_ALL}")

    tokenizer = WordPunctTokenizer()
    tokenized_sentences = []

    for sentence in sentences:
        #word_tokens = word_tokenize(sentence)
        word_tokens = tokenizer.tokenize(sentence)

        # stop_words  (filter tokeniz word by remove any stop word with except some word as from, where ... etc)
        filtered_tokens = filter_stop_words(word_tokens)

        tokenized_sentences.extend(filtered_tokens)
        #tokenized_sentences.append(word_tokens)


    # Get unique tokens ( Remove duplicates using set() and convert back to list )
    #print('len tokens before unique: ', len(tokenized_sentences))
    #print('1 tokens: ', tokenized_sentences )
    tokenized_sentences = list(set(tokenized_sentences))
    #print('len tokens after unique: ', len(tokenized_sentences))
    #print('2 tokens: ', tokenized_sentences )

    print("tokenized_sentences len:", len(tokenized_sentences) )
    print("tokenized_sentences:", tokenized_sentences)

    return tokenized_sentences




"""
  # Tf-idf Vectorizer (Term Frequency-Inverse Document Frequency Vectorizer)
  # Used to convert text data to numerical features
  # and get each one weight
  # care about frequency of a word within a sentence (term frequency)
  # and its rarity across the entire dataset (inverse document frequency)
"""
def Tfidf_Vectorizer(tokenized_sentences, X_train, X_test):
    print(f"{TextStyle.BOLD}{TextStyle.BLACK}------------ TfidfVectorizer ------------{TextStyle.RESET_ALL}")
    print('len tokenized_sentences -- : ', len(tokenized_sentences))
    tfidf_vectorizer = ''
    tfidf_vectorizer = TfidfVectorizer(max_features=5000)
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_train_tfidf_selected_features = tfidf_vectorizer.get_feature_names_out()

    X_test_tfidf = tfidf_vectorizer.fit_transform(X_test)
    X_test_tfidf_selected_features = tfidf_vectorizer.get_feature_names_out()

    # Checking the shape of the output matrices
    print('Shape of X_train_tfidf : ', X_train_tfidf.shape)
    print('Shape of X_test_tfidf : ', X_test_tfidf.shape)

    #Convert the first 5 vectors of the training set into a Dense DataFrame for better readability
    df_X_train_tfidf = pd.DataFrame(X_train_tfidf.toarray(), index = X_train, columns=X_train_tfidf_selected_features)
    df_X_test_tfidf  = pd.DataFrame(X_test_tfidf.toarray(), index = X_test, columns=X_test_tfidf_selected_features)
    print(df_X_train_tfidf)

    return X_train_tfidf, X_test_tfidf, df_X_train_tfidf, df_X_test_tfidf




def one_hot_decoder(df_data, tokenized_sentences):
    # One-hot encoding (using scikit-learn's OneHotEncoder)
    print(f"{TextStyle.BOLD}{TextStyle.BLACK}------------ OneHotEncoder ------------{TextStyle.RESET_ALL}")
    encoder_x = OneHotEncoder(sparse_output=False, categories=[tokenized_sentences])  # Dense output for easier visualization  (sparse to sparse_output to avoid warning remove/depracate)
    encoded_sentences = []
    encoded_sentences = encoder.fit_transform([[token in sentence for token in tokenized_sentences]
                                              for sentence in df_data['sentence']])

    # Get feature names
    tokens_names = encoder.get_feature_names_out(input_features=tokenized_sentences)

    print('tokens_names : ')
    print(tokens_names)

    # Convert encoded data to DataFrame with column names
    encoded_df = pd.DataFrame(encoded_sentences, columns=tokens_names)

    # Add sentence as header of row
    encoded_df.index = sentences

    print('encoded_sentences : ')
    print(encoded_sentences)

    print('------------------------')
    print('encoded_df : ')
    print(encoded_df)

    return encoded_df


### **performs feature selection using PCA (not used yet)**

In [44]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

"""
performs feature selection using Principal Component Analysis (PCA)
Args:
  data: X , which is array value of (num_rows, features)
  n_components : cols num. without dropd y column (column features)
  dataset_name : just name of dataset file
Returns:
  selected features df
"""
def perform_pca(data, n_components,dataset_name):
    # Extract feature names
    feature_names = data.columns.tolist()
    #Scaling Features
    print("Scaling Features using 'StandardScaler' from scikit-learn")

    """ Start net code """
    # Standardize features (optional, but recommended)
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)

    # Perform PCA
    pca = PCA(n_components=n_components)  # Number of components initially set to all features
    pca_result = pca.fit(data_scaled)
    """ End net code """

    """ Start Teacher code """
    #data_scaled = pd.DataFrame(preprocessing.scale(data),columns = data.columns)
    #pca = PCA(n_components=n_components)
    #pca_result = pca.fit_transform(data_scaled)
    """ End Teacher code """


    # Calculate explained variance ratio and cumulative variance
    explained_variance_ratio = pca.explained_variance_ratio_
    cumulative_variance      = np.cumsum(explained_variance_ratio)    # Calculates the cumulative explained variance ratio for each component.

    """  Start net code """
    # Determine the number of components based on explained variance ratio
    total_variance = np.sum(pca.explained_variance_)
    explained_variance_ratio = cumulative_variance / total_variance
    num_components = np.searchsorted(explained_variance_ratio, explained_variance_ratio) + 1

    # Select features based on explained variance ratio threshold
    selected_features = pca.components_[:, :num_components]
    print(f"{TextStyle.BOLD}Selected_features:{TextStyle.RESET_ALL} \n")

    # Transform data using selected components
    X_reduced = pca.transform(data_scaled)[:, :num_components]
    print('X_reduced : ', X_reduced)
    """ End net code """

    """ start Teacher code """
    # Get the selected feature names (corresponding to the original columns with highest loadings)
    selected_features_order = pd.DataFrame(pca.components_, columns=feature_names)
    selected_features_order = selected_features_order.abs().idxmax(axis=1)
    selected_features_order = selected_features_order.drop_duplicates()

    # keep 97%+ of the variance.
    number_of_cumulative_variance = 1 # init the number of components to keep
    vsum = 0 # to sum variance for each component (sumcum)
    for n in explained_variance_ratio:
      if vsum < 0.98:
        number_of_cumulative_variance += 1
      vsum += n
    selected_features = selected_features_order[:number_of_cumulative_variance]
    print("Keep 97%+ of the variance and thus keep " + str(number_of_cumulative_variance) + " Principal components from " + str(n_components) + ".\n")

    print(f"{TextStyle.BOLD}Selected_features:{TextStyle.RESET_ALL} \n")
    print(selected_features[:number_of_cumulative_variance +1])

    """ End Teacher code """


    # Create a DataFrame for PCA results
    pca_df = pd.DataFrame(data=pca_result, columns=feature_names)
    final_df = pca_df[selected_features]
    final_df.reset_index(inplace=True, drop=True)


    # Plot component variance and cumulative variance
    plt.figure(figsize=(12, 4))

    # Plot component variance with percentages
    plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio * 100, label='Component Variance')
    plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance * 100, marker='o', color='r', label='Cumulative Variance')

    plt.title('Variance Explained by Principal Components\n Dataset: ' + dataset_name)
    plt.xlabel('Principal Component')
    plt.ylabel('Percentage of Variance Explained (%)')
    plt.legend()
    plt.grid(True)
    plt.show()


    return final_df

### **Pre-process function**

In [58]:

def preprocess_data(df_data, class_Col, name):   # class_col ....
  print(f"{TextStyle.BOLD}{TextStyle.BLACK}------------ Pre-Proccess Data ------------{TextStyle.RESET_ALL}")


  ## Remove rows with missing target.
  #//print(f"{TextStyle.BOLD}{TextStyle.BLACK}------------ Handling Missing Target ------------{TextStyle.RESET_ALL}")
  #//print("Remove rows with missing target, separate target from predictors")


  # ----------- lowercase + strip 'sentence' ------------- #
  # Convert to lowercase & Remove spaces in start and end
  df_data['sentence'] = df_data['sentence'].str.lower().str.strip()

  # Remove accents (optional)
  # df_data['sentence'] = ''.join(x for x in unicodedata.normalize('NFKD', df_data['sentence']) if x in string.ascii_letters or x == " ")


  # ----------- Tokenization ------------- #
  tokenized_sentences = tokenization_text(df_data['sentence'])


  # ----------- OneHotEncoder ------------- #
  # encoded_df = one_hot_decoder(df_data, tokenized_sentences )



  #separate target from predictors
  n_classes = df_data[class_Col].nunique()    #no. classes/category in class_Col (attack_type)

  y = df_data[class_Col]
  y_attack_type = np.unique(y).tolist()

  # remove y column (class_Col) from column list
  #//df_data.drop([class_Col], axis=1, inplace=True)

  X = df_data.copy()
  X_columns = X.columns       #['sentence', 'len_payload']
  x_sentence_data = X['sentence']
  #print('X_columns : ', X['sentence'])


  #print('------------ 4 --------------')
  print(" X: ", X.shape)
  print(" y: ", y.shape)

  #change components number in case new columns add after preprossing
  n_components = X.shape[1]      # cols num. without dropd y column
  print('n_components : ', n_components)


  # ----------- PCA for feature selection ------------- #



  # ----------- train_test_split ------------- #
  # Divide the dataset into training (70%) and testing (30%)
  print(f"{TextStyle.BOLD}{TextStyle.BLACK}---------------- Divide the dataset into training (70%) and testing (30%) ----------------{TextStyle.RESET_ALL}")
  #### selected_features_df, y_encoder
  selected_features_df = df_data['sentence']  #x_sentence_data
  y_encoder = y  #_attack_type
  X_train, X_test, y_train, y_test = train_test_split(selected_features_df, y_encoder, test_size=0.3, random_state=42,stratify=y)
  print(f"X_train shape: {X_train.shape}")
  print(f"y_train shape: {y_train.shape}")
  print(f"X_test shape : {X_test.shape}")
  print(f"y_test shape : {y_test.shape}")


  # ----------- TfidfVectorizer ------------- #
  X_train_tfidf, X_test_tfidf, df_X_train_tfidf, df_X_test_tfidf = Tfidf_Vectorizer(tokenized_sentences, X_train, X_test)



  return n_classes, X_train, X_test, y_train, y_test, X_train_tfidf, X_test_tfidf, df_X_train_tfidf, df_X_test_tfidf




"""
## ------------- PCA for feature selection ------------------
## performs feature selection using Principal Component Analysis (PCA)
print(f"{TextStyle.BOLD}{TextStyle.BLACK}-------------- Principal Component Analysis (PCA) --------------{TextStyle.RESET_ALL}")
selected_features_df = perform_pca(X,n_components=n_components,dataset_name=name)
columns_witout_taget = selected_features_df.columns
#change components number in case new columns add after preprossing
n_components = selected_features_df.shape[1] - 1
# Encode categorical labels using one-hot encoding
print(f"{TextStyle.BOLD}{TextStyle.BLACK}Encode categorical labels using Label Encoder {TextStyle.RESET_ALL}")
# Initialize OneHotEncoder
encoder = LabelEncoder()
# Fit and transform the target column 'y'
y_encoded = encoder.fit_transform(y)
print(np.unique(y_encoded))
"""


""" ---------------- Apply --------------------
# Read Data of CSV Files
dataset_directory = "/content/datasets"   #files_path
percent = 100
data = read_csv_files(dataset_directory,percent)
print('len data : ' , len(data))


# Apply pre-processing
class_Col = 'attack_type'
n_classes, X_train, X_test, y_train, y_test, X_train_tfidf, \
X_test_tfidf, df_X_train_tfidf, df_X_test_tfidf = preprocess_data(data, class_Col, 'csv files')
"""


# Print results (optional)
#print('======================')
#for i, sentence in enumerate(sentences):
#  print(f"Sentence {i+1}:", sentence)
#  print("Tokens:", tokenized_sentences[i])
#  print("One-hot encoded:", encoded_sentences[i])
#  print("-" * 20)


------------ Handling Read CSV Files ------------
Your files are:  ['SQLiDataset_3_test.csv']
File Path:  /content/datasets/SQLiDataset_3_test.csv
Total rows in df/file:  89
Total rows in df/file 100%:  89
                                             sentence  attack_type  \
48                  end and   (  (   ( "poee" = "poee            1   
86                          stiefel8@spainbareback.eu            0   
53  -5596'  )  )   )  union all select 4877,4877,4...            1   
74                                         or a  =  a            1   
34   ) ; IF ( 3509 = 3509 )  SELECT 3509 ELSE DROP...            1   
..                                                ...          ...   
58                                   blakley@asyra.ao            0   
66                                           or 1 = 1            1   
71                                          or 1 = 1#            1   
82  s:,s: select * from users where id = 1 or ":,"...            1   
33           ) ; BEGIN D

## **create_generator**

The model of the generator is a deep multi-layer perceptron (MLP) composed of an input layer, output layer and four hidden layers. Initially, the generator takes a point from the latent space to generate new data. The latent space is a multi-dimensional hypersphere normal distributed points, where each variable is drawn from the distribution of the data in the dataset. An embedded layer in the generator creates a vector representation for the generated point.
The generator model has four hidden layers. The first one is composed of 256 neurons with a rectified linear unit (ReLU) activation function. An embedded layer is used between hidden layers to efficiently map input data from a high-dimension to lower-dimension space. This allows the neural network to learn the data relationship and process it efficiently. The second hidden layer consists of 128 neurons, the third has 64 neurons and the last one has 32 neurons, with the ReLU activation function used with them all, and a regularization dropout of 20% is added to avoid overfitting. The output layer is activated using the Softmax activation function with 14 neurons as the number of features in the dataset.

In [60]:
"""
Creates a generator model for text generation using a deep MLP.

Args:
    latent_dim: The dimension of the latent space.
    vocabulary_size: The size of the vocabulary (number of unique words).  ((get it by tokeniz))

Returns:
    A TensorFlow Keras model representing the text generation GAN generator.
"""

def create_generator(latent_dim, vocabulary_size):

  # Input layer for latent space noise
  noise = Input(shape=(latent_dim,), name='noise')

  # Embedded layer to map noise to lower-dimensional space
  # It creates a vector representation for the generated point.
  hidden = Embedding(latent_dim, vocabulary_size // 2)(noise)  # Adjust embedding dim as needed

  # Hidden layers with ReLU activation and dropout
  hidden = Dense(256, activation='relu')(hidden)
  hidden = Dropout(DISCRIMINATOR_DROPOUT_RATE)(hidden)        # DISCRIMINATOR_DROPOUT_RATE = 0.3

  hidden = Dense(128, activation='relu')(hidden)
  hidden = Dropout(DISCRIMINATOR_DROPOUT_RATE)(hidden)        # DISCRIMINATOR_DROPOUT_RATE = 0.3

  hidden = Dense(64, activation='relu')(hidden)
  hidden = Dropout(DISCRIMINATOR_DROPOUT_RATE)(hidden)        # DISCRIMINATOR_DROPOUT_RATE = 0.3

  hidden = Dense(32, activation='relu')(hidden)
  hidden = Dropout(DISCRIMINATOR_DROPOUT_RATE)(hidden)        # DISCRIMINATOR_DROPOUT_RATE = 0.3

  # Output layer with Softmax activation
  generated_text = Dense(vocabulary_size, activation='softmax', name='generated_text')(x)

  # Create the generator model
  generator = Model(inputs=noise, outputs=generated_text, name='generator')

  return generator

## **create_discriminator**

In the TDCGAN architecture consists of three discriminators. Each discriminator is a deep neural network with different architecture and different parameter settings. Each discriminator’s role is to extract features from the output of the generator and classify the data with varying levels of accuracy for each them. The model aims to classify data into two groups: normal flows for the background traffic with 0 representation, and anomaly flows for the attack data with 1 representation. The discriminator distinguishes the new data generated by the generator from the true data distribution. It classifies them as either real or fake. Subsequently, the discriminator undergoes updates to improve its ability to distinguish between real and fake samples in the subsequent round. The discriminators try to classify the data into their corresponding class, which is done through a fully connected MLP network.

Each discriminator is a MLP model with a different number of hidden layers, different number of neurons and different dropout percentage. The first discriminator is composed of 3 hidden layers with 100 neurons for each and 10% dropout regularization. The second has five hidden layers with 64, 128, 256, 512, and 1024 neurons for each layer, respectively. The dropout percentage is 40%. The last discriminator has 4 hidden layers with 512, 256, 128, and 64 neurons for each layer and 20% dropout percentage.
The LeakyReLU(alpha = 0.2) is used as an activation function for the hidden layers in the discriminators. Two output layers are used for each discriminator with the Softmax function as an activation function for one output layer and the Sigmoid activation function for the second output layer.

***Why 3 discriminator?***

In this model, three discriminators are used and each discriminator haas different architecture. These considered a modified training strategy which helped to face challenge in detection tasks. So, it help the generator to reach its optimal state even when the discriminator converges quickly during the initial stages of training.


In [5]:
"""
Creates a discriminator model for TDCGAN with specified architecture.

Args:
    data_dim          : Dimensionality of the data features (based on your data size)
    num_classes       : Number of classes in the dataset (maybe - num_attack_classes)
    name              : Name for the discriminator model.
    num_hidden_layers : Number of hidden layers in the discriminator.
    neurons_per_layer : List of integers specifying the number of neurons in each hidden layer.
    dropout_rate      : Dropout rate for regularization.

Returns:
    A TensorFlow Keras model representing the TDCGAN discriminator.
"""

def create_discriminator(data_dim, num_classes, name, num_hidden_layers, neurons_per_layer, dropout_rate):

  # Input layers for data (real or generated) and label
  # our data is Conditional Generation so label is needed
  data  = Input(shape=(data_dim,), name='data')       # Features    (ex: 100 samples, 10 features)
  label = Input(shape=(num_classes,), name='label')   # One-hot encoded label


  # Concatenate features and label  (Concatenate along the last dimension)
  """  Concatenation: A Concatenate layer is used to combine the features and label along the last dimension (axis=-1).
      This creates a single input for the subsequent hidden layers in the discriminator model."""
  concatenated = Concatenate(axis=-1)([data, label])   #Concatenate(axis=-1)


  # Hidden layers with LeakyReLU activation and dropout
  hidden = data
  for i in range(num_hidden_layers):
    if i == 0:
      hidden = Dense(neurons_per_layer[i], activation='leaky_relu', alpha=0.2)(concatenated)
      hidden = Dropout(dropout_rate)(hidden)
    else:
      hidden = Dense(neurons_per_layer[i], activation='leaky_relu', alpha=0.2)(hidden)
      hidden = Dropout(dropout_rate)(hidden)
      hidden = BatchNormalization()(hidden)


  # Output layer for real vs fake prediction (Real/fake classification)
  validity_output = Dense(1, activation='sigmoid', name='validity_output')(hidden)

  # Output layer for class label prediction (normal/sqli attack classification)   (Auxiliary classification / class_output)
  auxiliary_output = Dense(num_classes, activation='softmax', name='auxiliary_output')(hidden)

  # Build and compile the discriminator model
  discriminator = Model(inputs=[data, label], outputs=[auxiliary_output, validity_output], name=name)    # inputs=data

  # auxiliary_output (attack_type maybe) may binary_crossentropy  (0 --> normal / 1 --> SQLi)
  discriminator.compile(
      loss={'auxiliary_output': 'binary_crossentropy', 'validity_output': 'binary_crossentropy'},    #'auxiliary_output': 'categorical_crossentropy',
      optimizer=Adam(learning_rate= '0.0002' , beta_1= '0.5'),
      metrics={'auxiliary_output': 'accuracy', 'validity_output': 'accuracy'})

  return discriminator



""" ----------------- Apply Discriminater with req. values ------------------ """
# Define discriminator architectures
#discriminator_1 = create_discriminator(2, 'discriminator_1', 3, [100, 100, 100], 0.1)
#discriminator_2 = create_discriminator(2, 'discriminator_2', 5, [64, 128, 256, 512, 1024], 0.4)
#discriminator_3 = create_discriminator(2, 'discriminator_3', 4, [512, 256, 128, 64], 0.2)

"""
# May used in training as EX
# Example Data (assuming you have 3 classes and data_dim is 10)
real_data = tf.random.normal(shape=(100, 10))  # Sample real data (100 samples, 10 features)
label_one_hot = tf.one_hot(tf.random.uniform(shape=(100,), minval=0, maxval=num_classes, dtype=tf.int32), depth=num_classes)  # One-hot encoded labels (100 samples)
"""


' ----------------- Apply Discriminater with req. values ------------------'

## **Train**

Through training, the generator learns to map points from the latent space into specific output data, which are different each time the model is trained. Taken a step further, new data are then generated using random points in the latent space. So, these points are used to generate specific data. The discriminator distinguishes the new data generated by the generator from the true data distribution.
Here both the generator and discriminator models are trained simultaneously and conditioned on the class label. This conditioning enables the generator model, when utilized independently, to generate minor class data within the domain that corresponds to a specific class label. Subsequently, the discriminator undergoes updates to improve its ability to distinguish between real and fake samples in the subsequent round. Additionally, the generator receives updates based on its success or failure in deceiving the discriminator with its generated samples.
To Ensure a more balanced training process between the generator and discriminators by three discriminators with different architectures.
In this manner, the two models engage in a competitive relationship, exhibiting adversarial behavior in the context of game theory. In this scenario, the concept of zero-sum implies that when the discriminator effectively distinguishes between real and fake samples, it receives a reward, or no adjustments are made to its model parameters. Simultaneously, the generator is penalized with significant updates to its model parameters. Alternatively, when the generator successfully deceives the discriminator, it receives a reward, or no modifications are made to its model parameters. Whereas, the discriminator is penalized. This is the generic GAN approach.
The model is trained for 1000 epochs with a batch size of 128. The optimizer is Adam with a learning rate equal to 0.0001. The proposed model allows the generator to train until it produces a new set of data samples that resembles the real distribution of the original dataset.

The discriminators undergo separate training, where each of the model weights are designated as non-trainable within the TDCGAN model. This ensures that solely the weights of the generator model are updated during the training process. This trainability modification specifically applies when training the TDCGAN model, not when training the discriminator independently. So, the TDCGAN model is employed to train the generator’s model weights by utilizing the output and error computed by the discriminator models.

The primary objective of the training methodology employed in the GAN framework is for the generator to generate fake data that closely resemble real data, and for the discriminator to acquire sufficient knowledge to differentiate between real and fake samples. Both the generator and discriminator are trained until the discriminator can no longer distinguish real data from fake data. This mean that the generated network can estimate the data sample’s distribution and achieve Nash equilibrium.In order to assess the performance of our model with precision, it is customary to divide the data into training and test sets to produce accurate predictions on unseen data.
The training set is utilized for model fitting, while the test set is employed to measure the predictive precision of the trained model. The dataset is split into 70% for training and validation and 30% for testing. The training set is divided into minor class data and other class data. The TDCGAN model uses the minor class to generate data. The generator is trained to model the distribution of the anomaly data (minor class), while fixing the discriminator. The output from the generator is fed as input to the discriminator to predict it.
The error is estimated, and the generator’s weight is then updated. The training continues until the discriminator cannot distinguish if the input data come from the generator’s output or from the real anomaly dataset. In the training process, we make sure that all architectures undergo an equal number of epochs and that the weights from the final epoch are selected to generate artificial attack samples.
We begin by adhering to this iterative training procedure and ultimately utilize the generator to produce attack samples. Eventually, we incorporate the generated attack samples into the training set. By this, we oversample minor classes in the dataset during the training phase. The test dataset is then used to test the model performance.


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist  # Replace with your dataset

# Hyperparameters
epochs = 1000
batch_size = 128
learning_rate = 0.0001
split_ratio = 0.7  # 70% for training and validation, 30% for testing

# Load data (replace with your data loading logic)
(x_train, y_train), (_, _) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28)  # Flatten images (adjust based on data shape)
x_train = x_train.astype('float32') / 255.0  # Normalize data

# One-hot encode labels (adjust num_classes if needed)
num_classes = 10  # Assuming 10 classes
y_train = tf.one_hot(y_train, depth=num_classes)

# Split data into training, validation, and test sets
train_val_split = int(split_ratio * len(x_train))
x_train, x_val, y_train, y_val = x_train[:train_val_split], x_train[train_val_split:], y_train[:train_val_split], y_train[train_val_split:]
x_test, y_test = x_train[train_val_split:], y_train[train_val_split:]

# Define functions for generator and discriminators (replace with your architectures)
def create_generator(latent_dim, num_classes):
    # ... your generator architecture here ...
    return model

def create_discriminator(data_dim, num_classes, name):
    # ... your discriminator architecture here ...
    return model

# Create generator and discriminators
generator = create_generator(latent_dim=100, num_classes=num_classes)  # Adjust latent space dimension
discriminator_1 = create_discriminator(data_dim=28 * 28, num_classes=num_classes, name='discriminator_1')
discriminator_2 = create_discriminator(data_dim=28 * 28, num_classes=num_classes, name='discriminator_2')
discriminator_3 = create_discriminator(data_dim=28 * 28, num_classes=num_classes, name='discriminator_3')

# Define loss functions (replace with appropriate loss functions)
generator_loss_function = tf.keras.losses.BinaryCrossentropy(from_logits=True)
discriminator_loss_function = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Optimizers for generator and discriminators
generator_optimizer = Adam(learning_rate=learning_rate)
discriminator_optimizer = Adam(learning_rate=learning_rate)

# Training loop
for epoch in range(epochs):
    # Train generator with all discriminators fixed
    for _ in range(1):  # Train generator once per epoch (adjust as needed)
        noise = tf.random.normal(shape=(batch_size, latent_dim))  # Sample random noise
        label = tf.random.uniform(shape=(batch_size,), minval=0, maxval=num_classes, dtype=tf.int32)  # Random class labels
        label = tf.one_hot(label, depth=num_classes)

        with tf.GradientTape() as gen_tape:
            generated_data = generator([noise, label])
            real_output = tf.concat([tf.ones_like(generated_data[:, 0:1]), tf.zeros_like(generated_data[:, 1:])], axis=1)  # Expected output for real data (all ones for validity)
            for discriminator in [discriminator_1, discriminator_2, discriminator_3]:
                discriminator_output = discriminator([generated_data, label])
                gen_loss = generator_loss_function(real_output, discriminator_output[0])  # Consider combining losses from all discriminators
                gen_tape.gradient(gen_loss, generator.trainable_variables)

        generator_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
        generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))

    # Train each discriminator separately with generator fixed

## **Run code TDCGAN Model**

### **run_tdcgan**

In [ ]:
def run_tdcgan(X_t, y_t,encoder,name,class_Col,n_classes,features_dim, latent_dim):
  # Create a DataFrame
  df = pd.DataFrame(data=X_t)
  cols = df.columns
  df[class_Col] = y_t

  # Identify the majority class and count
  majority_class = df[class_Col].value_counts().idxmax()
  majority_count = len(df[df[class_Col] == majority_class])

  # Select only the minority classes data for training TDCGAN
  minority_data = df[~df[class_Col].isin([majority_class])]
  labels_dic = (minority_data[class_Col].value_counts()).to_dict()

  model = TDCGAN(latent_dim,n_classes -1,features_dim ,features_dim)
  model.print_parameters_and_summary()
  model.plot_architecture()

  # train model (minority_data train / unbalanced)
  y_tr = minority_data[class_Col]
  minority_data.drop([class_Col], axis=1, inplace=True)
  model.train(minority_data, y_tr, " unbalanced " + name)

  # Save the trained model
  print("Saving model ")
  save_model_2(model,name,'tdcgan')

  #Generate synthetic samples for the minority classes
  print(f"{TextStyle.BOLD}Generate synthetic features and labels{TextStyle.RESET_ALL}")
  for key, value in labels_dic.items():
    labels_dic[key]= majority_count - value
  synthetic_data = model.generate_synthetic_data(labels_dic,cols,class_Col)

  # Concatenate synthetic samples with the original dataset
  fdf = pd.concat([df, synthetic_data], ignore_index=True, sort=False)

  # Count the occurrences of each class in the original labels
  unique_classes, original_counts = np.unique(encoder.inverse_transform(y_t), return_counts=True)

  # Count the occurrences of each class in the balanced labels
  unique_resampled_classes, resampled_counts = np.unique(encoder.inverse_transform(fdf[class_Col]), return_counts=True)

  # Sort the data from largest to smallest
  original_counts, unique_classes = zip(*sorted(zip(original_counts, unique_classes), reverse=True))
  #resampled_counts, unique_resampled_classes = zip(*sorted(zip(resampled_counts, unique_resampled_classes), reverse=True))
  print('Original Data class counter:\n', Counter(encoder.inverse_transform(y_t)))
  print('Balanced Data class counter:\n', Counter(encoder.inverse_transform(fdf[class_Col])))

  # Create bar positions
  x = np.arange(len(unique_classes))

  # Set the width of the bars
  width = 0.4
  # Create the figure and axis objects
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
  # Plot the original class distribution
  ax1.bar(x, original_counts, width, color=colors_list)
  ax1.set_xlabel('Classes')
  ax1.set_ylabel('Counts')
  ax1.set_title('Original Class Distribution\n Dataset ' + name + "\n" + str(X_t.shape))
  ax1.set_xticks(x)
  ax1.set_xticklabels(unique_classes,rotation='vertical')
  #ax1.legend()

  # Plot the balanced class distribution
  ax2.bar(x, resampled_counts, width, color=colors_list)
  ax2.set_xlabel('Classes')
  ax2.set_ylabel('Counts')
  ax2.set_title('Balanced Class Distribution ' + "TDCGAN" + '\n Dataset ' + name + "\n" + str(fdf.shape))
  ax2.set_xticks(x)
  ax2.set_xticklabels(unique_classes,rotation='vertical')
  #ax2.legend()

  plt.tight_layout()
  plt.show()
  return fdf

### **Balance Dataset**

Learning from imbalanced dataset is one of the main challenges that ML facing and it's a common problem. Because collecting balanced dataset cann't be guaranteed in many different fields. For example, in the website and application security field that software engineers face, more normal information is usually entered on the website than malicious attacks, which generates unbalanced data. This problem can be solved by balancing the data through reducing the majority classes and increasing the minority classes.

***Check is balanced***

In [67]:
import pandas as pd
from collections import Counter

def check_data_isbalance(data, class_Col):
    df1 = pd.DataFrame(data)
    cat_class = df1[class_Col]

    # Count the occurrences of each value in class_Col
    count_per_value = Counter(cat_class)

    # Check if the counts of the unique values are equal
    are_counts_equal = len(set(count_per_value.values())) == 1

    print("Count of rows for each value in class_Col:")
    for value, count in count_per_value.items():
        print(f"Value {value}: {count} rows")

    if are_counts_equal:
        print("The counts of values in class_Col are equal.")
    else:
        print("The counts of values in class_Col are not equal.")

    return count_per_value, are_counts_equal

data = {
        'sentence' : [ "' or 1=1" , 'hi soma', '29%', '--', 'nice'],
        'attack_type' : [1, 0, 1, 1, 0]
}

class_Col = 'attack_type'
count_per_value, are_counts_equal = check_data_isbalance(data, class_Col)


Count of rows for each value in class_Col:
Value 1: 3 rows
Value 0: 2 rows
The counts of values in class_Col are not equal.


***Read random n rows from dataset***

In [ ]:
""" ----------------- Read Random n rows Function ----------------- """
def read_random_n_rows(df, nrows, percent):
    # Check if requested sample size is greater than the total number of rows
	  # nrows --> requested sample size
	  # len(df) --> total number of rows
    if nrows >= len(df):
        return df

    # Randomly select n rows from the DataFrame
    random_sample = df.sample(n=nrows, random_state=42)
    random_sample.reset_index(drop=True,inplace=True)
    return random_sample


""" ----------------- Apply Read Random n rows Function ----------------- """
#nrows = 12000
#random_sample = read_random_n_rows(data, nrows, percent)
#random_sample

***Balancing dataset***

In [ ]:


def balancing_data(file_name, data, percent, class_Col):
    path = '/content/drive/MyDrive/code/ids/CIC-IDS2017'
    percent =  100 #100% from each file (8 files)
    data = read_csv_files(path,percent)
    data = upclasses(data,' Label',2000)
    data[' Label'].value_counts()

    #save data to file
    path='/content/drive/MyDrive/code/ids/CIC-IDS2017-NEW/'
    data.to_csv(path + 'CIC-IDS2017.csv', index=False)


    """**Selecting subset of 60 from each dataset**"""

    #Read 60 raw from each dataset and save each new 200,000 raw dataset in a new file.
    percent =  100 #100% from each file (8 files)
    nrows = 60
    path = '/content/drive/MyDrive/code/ids/CIC-IDS2017-NEW'
    print("\nSubset of 200,000 from each dataset \n ")
    #read 200,000 recored from CIC-IDS2017 without selected rows
    data = read_random_n_rows(path + "/CIC-IDS2017.csv", nrows,percent)

    #save data to file
    path='/datasets/balanced/'
    data.to_csv(path + 'CIC-IDS2017.csv', index=False)
    #plot the data
    fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(20,5))
    colors_list = ['Red','Orange', 'Blue', 'Purple','Green','Pink','Gray','Tan','Lime','Cyan']
    r1 = data[' Label'].value_counts().plot(kind='bar',color=colors_list,ax=ax[0])
    r1.set_xticklabels(r1.get_xticklabels())
    r1.set_title(str(nrows) + " of the CIC_IDS2017 dataset\n" + str(data.shape))
    for tick in r1.get_xticklabels():
        tick.set_rotation(90)
    for rect in r1.patches:
        r1.text (rect.get_x() + rect.get_width()  / 2,rect.get_height()+ 0.75, "  " + str(round(rect.get_height()/len(data) * 100,4)) + "%",rotation='vertical', color='darkolivegreen', fontsize = 7)



""" ----------------- Apply Read Random n rows Function ----------------- """
nrows = 60
percent = 100

print("\nSubset of " , nrows , " from each dataset \n ")
#read nrows recored from dataset file without selected rows
random_sample = read_random_n_rows(data_readed_file, nrows, percent)
random_sample

### **Run Code**

In [43]:

""" ----------------- Apply Code ----------------- """
# Read Data of CSV Files
dataset_directory = "/content/datasets"   #files_path
percent = 100
data = read_csv_files(dataset_directory,percent)
print('len data : ' , len(data))


# Apply pre-processing
class_Col = 'attack_type'
n_classes, X_train, X_test, y_train, y_test, X_train_tfidf, \
X_test_tfidf, df_X_train_tfidf, df_X_test_tfidf = preprocess_data(data, class_Col, 'csv files')


#data = run_tdcgan(X_train.copy(), y_train.copy(),encoder,name,class_Col,n_classes,X_train.shape[1], 100)


------------ Handling Read CSV Files ------------
Your files are:  ['SQLiDataset_3_test.csv']
File Path:  /content/datasets/SQLiDataset_3_test.csv
Total rows in df/file:  89
Total rows in df/file 100%:  89
                                             sentence  attack_type  \
28   ( select * from  ( select ( sleep ( 5  )  )  ...            1   
50   Experts say violence left 14 adults seven chi...            0   
53  -5596'  )  )   )  union all select 4877,4877,4...            1   
25   &quot; Mr Edis said Coulson &apos;s involveme...            0   
7                                            12:00 PM            0   
..                                                ...          ...   
88                 ")) or benchmark(10000000,MD5(1))#            1   
85                    stief.kenworthy@todomodelos.edu            0   
12                                                  5            0   
75                                           or a = a            1   
4                       

## **Test 1**

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Conv1D,
    Flatten,
    Dense,
    Embedding,
    Input,
    LeakyReLU,
    BatchNormalization,
    Reshape,
)
from tensorflow.keras.optimizers import Adam

# Hyperparameters (adjust as needed)
vocab_size = 10000  # Adjust based on your vocabulary
embedding_dim = 128
noise_dim = 100
latent_dim = 100  # Dimension of latent space for conditional generation

def preprocess_text(text):
  """
  Preprocesses a text string for use in the DCGAN model.

  Args:
      text: The input text string.

  Returns:
      A list of preprocessed tokens.
  """
  # ... (same preprocessing steps from previous response)

# Text embedding layer
embedding_layer = Embedding(vocab_size, embedding_dim, input_length=max_sentence_length)


# Create separate models for generator and discriminator
generator = build_generator(noise_dim, latent_dim, embedding_dim)

In [28]:
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer

def preprocess_sentences(sentences):
  """
  Preprocesses a list of sentences into one-hot encoded data.

  Args:
      sentences: A list of strings, where each string represents a sentence.

  Returns:
      A tuple containing:
          - sequences: A list of lists, where each inner list represents the one-hot encoded word indices for a sentence.
          - word_index: A dictionary mapping each unique word to its unique index.
  """

  # Count word occurrences
  word_counts = Counter()
  for sentence in sentences:
    words = sentence.lower().split()  # Lowercase and split into words
    word_counts.update(words)

  # Filter out infrequent words (optional)
  min_count = 5  # Adjust minimum count threshold as needed
  word_counts = {word: count for word, count in word_counts.items() if count >= min_count}

  # Create tokenizer and fit on word counts
  tokenizer = Tokenizer(num_words=len(word_counts) + 1)  # +1 for padding
  tokenizer.fit_on_texts(word_counts.keys())

  # Convert sentences to sequences of word indices
  sequences = tokenizer.texts_to_sequences(sentences)

  # Pad sequences to the same length (optional)
  max_len = 20  # Adjust maximum sentence length as needed
  from tensorflow.keras.preprocessing.sequence import pad_sequences
  sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

  # Create one-hot encoded representation
  one_hot_encoded = np.array([tf.one_hot(seq, len(word_counts) + 1) for seq in sequences])

  return one_hot_encoded, tokenizer.word_index

# Example usage
sentences = [
    "select * form dual where x=2 or 1=1",
    "update table_name set column1=value1 where condition",
    "insert into table_name (column1, column2) values ('value1', 'value2')",
]

one_hot_encoded, word_index = preprocess_sentences(sentences)

print("One-hot encoded data:", one_hot_encoded.shape)
print("Example sentence:", sentences[0])
print("Corresponding one-hot encoded sequence:", one_hot_encoded[0])
print("Word index:", word_index)

NameError: name 'np' is not defined

# **test 2**

In [17]:
import uuid

# Initialize variables
NUM_EPOCHS = 50
BUFFER_SIZE = 30000
BATCH_SIZE = 28
NOISE_DIMENSION = 75
UNIQUE_RUN_ID = str(uuid.uuid4())
PRINT_STATS_AFTER_BATCH = 50
OPTIMIZER_LR = 0.0002
OPTIMIZER_BETAS = (0.5, 0.999)
WEIGHT_INIT_STDDEV = 0.02

GENERATOR_DROPOUT_RATE = 0.1
DISCRIMINATOR_DROPOUT_RATE = 0.3      #Adjust the dropout rate to prevent overfitting during training.
LEAKY_RELU_ALPA = 0.2

In [23]:
import tensorflow # as tf

# Initialize loss function, init schema and optimizers
cross_entropy_loss = tensorflow.keras.losses.BinaryCrossentropy(from_logits=True)

weight_init = tensorflow.keras.initializers.RandomNormal(stddev=WEIGHT_INIT_STDDEV)

generator_optimizer = tensorflow.keras.optimizers.Adam(OPTIMIZER_LR, \
  beta_1=OPTIMIZER_BETAS[0], beta_2=OPTIMIZER_BETAS[1])

discriminator_optimizer = tensorflow.keras.optimizers.Adam(OPTIMIZER_LR, \
  beta_1=OPTIMIZER_BETAS[0], beta_2=OPTIMIZER_BETAS[1])

In [24]:
def make_directory_for_run():
  """ Make a directory for this training run. """
  print(f'Preparing training run {UNIQUE_RUN_ID}')
  if not os.path.exists('./runs'):
    os.mkdir('./runs')
  os.mkdir(f'./runs/{UNIQUE_RUN_ID}')

In [25]:
def generate_image(generator, epoch = 0, batch = 0):
  """ Generate subplots with generated examples. """
  images = []
  noise = generate_noise(BATCH_SIZE)
  images = generator(noise, training=False)
  plt.figure(figsize=(10, 10))
  for i in range(16):
    # Get image and reshape
    image = images[i]
    image = np.reshape(image, (28, 28))
    # Plot
    plt.subplot(4, 4, i+1)
    plt.imshow(image, cmap='gray')
    plt.axis('off')
  if not os.path.exists(f'./runs/{UNIQUE_RUN_ID}/images'):
    os.mkdir(f'./runs/{UNIQUE_RUN_ID}/images')
  plt.savefig(f'./runs/{UNIQUE_RUN_ID}/images/epoch{epoch}_batch{batch}.jpg')

In [26]:
#path='mnist.npz'
def load_data():
  """ Load data """
  (images, _), (_, _) = tensorflow.keras.datasets.mnist.load_data()
  images = images.reshape(images.shape[0], 28, 28, 1)
  images = images.astype('float32')
  images = (images - 127.5) / 127.5
  return tensorflow.data.Dataset.from_tensor_slices(images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [39]:
from tensorflow import keras
from tensorflow.keras.layers import (
    Conv2D,
    Flatten,
    Dense,
    LeakyReLU,
    Dropout,
    BatchNormalization,  # Consider adding for training stability
)

#GENERATOR_DROPOUT_RATE = 0.1

def create_generator():
  """ Create Generator """
  generator = tensorflow.keras.Sequential()

  # Input block
  generator.add(Dense(7*7*128, use_bias=False, input_shape=(NOISE_DIMENSION,), kernel_initializer=weight_init))
  generator.add(BatchNormalization())
  generator.add(LeakyReLU())

  # Reshape 1D Tensor into 3D
  generator.add(Reshape((7, 7, 128)))

  # First upsampling block
  generator.add(Conv2DTranspose(56, (5, 5), strides=(1, 1), padding='same', use_bias=False, kernel_initializer=weight_init))
  generator.add(BatchNormalization())
  generator.add(LeakyReLU())

  # Second upsampling block
  generator.add(Conv2DTranspose(28, (5, 5), strides=(2, 2), padding='same', use_bias=False, \
    kernel_initializer=weight_init))
  generator.add(BatchNormalization())
  generator.add(LeakyReLU())

  # Third upsampling block: note tanh, specific for DCGAN
  generator.add(Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh', \
    kernel_initializer=weight_init))

  # Return generator
  return generator

In [28]:
def generate_noise(number_of_images = 1, noise_dimension = NOISE_DIMENSION):
  """ Generate noise for number_of_images images, with a specific noise_dimension """
  return tensorflow.random.normal([number_of_images, noise_dimension])

In [38]:
def create_discriminator():
  """ Create Discriminator """
  discriminator = tensorflow.keras.Sequential()

  # First Convolutional block (CNN Layer 1)
  discriminator.add(Conv2D(28, (5, 5), strides=(2, 2), padding='same',    #(32, (3, 3), strides=(1, 1)
                                    input_shape=[28, 28, 1], kernel_initializer=weight_init))   #
  discriminator.add(LeakyReLU(alpha=LEAKY_RELU_ALPA))
  discriminator.add(Dropout(DISCRIMINATOR_DROPOUT_RATE))

  # Second Convolutional block (CNN Layer 2)
  discriminator.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same', kernel_initializer=weight_init))   #(3, 3)     kernel_initializer='he_normal'))
  discriminator.add(LeakyReLU(alpha=LEAKY_RELU_ALPA))
  discriminator.add(Dropout(DISCRIMINATOR_DROPOUT_RATE))
  discriminator.add(BatchNormalization())  # Consider adding for to improve training stability, especially for deeper networks.

  # Third Convolutional block (CNN Layer 3)
  discriminator.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same', kernel_initializer=weight_init))   #(3, 3)     kernel_initializer='he_normal'))
  discriminator.add(LeakyReLU(alpha=LEAKY_RELU_ALPA))
  discriminator.add(Dropout(DISCRIMINATOR_DROPOUT_RATE))
  discriminator.add(BatchNormalization())  # Consider adding for training stability

  # Flatten and generate output prediction
  # numerical representations of text (word embeddings) or flattened representations of images.
  discriminator.add(Flatten())
  discriminator.add(Dense(1, kernel_initializer=weight_init, activation='sigmoid'))  #relu    #kernel_initializer='he_normal'

  # Return discriminator
  return discriminator


In [29]:
def compute_generator_loss(predicted_fake):
  """ Compute cross entropy loss for the generator """
  return cross_entropy_loss(tensorflow.ones_like(predicted_fake), predicted_fake)


def compute_discriminator_loss(predicted_real, predicted_fake):
  """ Compute discriminator loss """
  loss_on_reals = cross_entropy_loss(tensorflow.ones_like(predicted_real), predicted_real)
  loss_on_fakes = cross_entropy_loss(tensorflow.zeros_like(predicted_fake), predicted_fake)  return loss_on_reals + loss_on_fakes

In [30]:
def save_models(generator, discriminator, epoch):
  """ Save models at specific point in time. """
  tensorflow.keras.models.save_model(
    generator,
    f'./runs/{UNIQUE_RUN_ID}/generator_{epoch}.model',
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
  )
  tensorflow.keras.models.save_model(
    discriminator,
    f'./runs/{UNIQUE_RUN_ID}/discriminator{epoch}.model',
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
  )

def print_training_progress(batch, generator_loss, discriminator_loss):
  """ Print training progress. """
  print('Losses after mini-batch %5d: generator %e, discriminator %e' %
        (batch, generator_loss, discriminator_loss))

In [31]:
@tensorflow.function
def perform_train_step(real_images, generator, discriminator):
  """ Perform one training step with Gradient Tapes """
  # Generate noise
  noise = generate_noise(BATCH_SIZE)
  # Feed forward and loss computation for one batch
  with tensorflow.GradientTape() as discriminator_tape, \
      tensorflow.GradientTape() as generator_tape:
        # Generate images
        generated_images = generator(noise, training=True)
        # Discriminate generated and real images
        discriminated_generated_images = discriminator(generated_images, training=True)
        discriminated_real_images = discriminator(real_images, training=True)
        # Compute loss
        generator_loss = compute_generator_loss(discriminated_generated_images)
        discriminator_loss = compute_discriminator_loss(discriminated_real_images, discriminated_generated_images)
  # Compute gradients
  generator_gradients = generator_tape.gradient(generator_loss, generator.trainable_variables)
  discriminator_gradients = discriminator_tape.gradient(discriminator_loss, discriminator.trainable_variables)
  # Optimize model using gradients
  generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))
  # Return generator and discriminator losses
  return (generator_loss, discriminator_loss)

In [32]:
def train_gan(num_epochs, image_data, generator, discriminator):
  """ Train the GAN """
  # Perform one training step per batch for every epoch
  for epoch_no in range(num_epochs):
    num_batches = image_data.__len__()
    print(f'Starting epoch {epoch_no+1} with {num_batches} batches...')
    batch_no = 0
    # Iterate over batches within epoch
    for batch in image_data:
      generator_loss, discriminator_loss = perform_train_step(batch, generator, discriminator)
      batch_no += 1
      # Print statistics and generate image after every n-th batch
      if batch_no % PRINT_STATS_AFTER_BATCH == 0:
        print_training_progress(batch_no, generator_loss, discriminator_loss)
        generate_image(generator, epoch_no, batch_no)
    # Save models on epoch completion.
    save_models(generator, discriminator, epoch_no)
  # Finished :-)
  print(f'Finished unique run {UNIQUE_RUN_ID}')

In [33]:
def run_gan():
  """ Initialization and training """
  # Make run directory
  make_directory_for_run()
  # Set random seed
  tensorflow.random.set_seed(42)
  # Get image data
  data = load_data()
  # Create generator and discriminator
  generator = create_generator()
  discriminator = create_discriminator()
  # Train the GAN
  print('Training GAN ...')
  train_gan(NUM_EPOCHS, data, generator, discriminator)

In [ ]:
if __name__ == '__main__':
  run_gan()

Preparing training run 56983348-bd6a-4e9b-a421-586f759b0dd5


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Training GAN ...
Starting epoch 1 with 469 batches...


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5818: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Losses after mini-batch    50: generator 6.083512e-01, discriminator 1.074799e+00
Losses after mini-batch   100: generator 9.137599e-01, discriminator 8.789127e-01
Losses after mini-batch   150: generator 1.509523e+00, discriminator 1.189188e+00
Losses after mini-batch   200: generator 9.621420e-01, discriminator 1.130656e+00
Losses after mini-batch   250: generator 7.280799e-01, discriminator 1.197280e+00
Losses after mini-batch   300: generator 8.209665e-01, discriminator 1.288694e+00
Losses after mini-batch   350: generator 8.899400e-01, discriminator 1.223952e+00
Losses after mini-batch   400: generator 8.486003e-01, discriminator 1.359012e+00
Losses after mini-batch   450: generator 8.501085e-01, discriminator 1.219294e+00


...

Starting epoch 44 with 469 batches...
Losses after mini-batch    50: generator 7.169980e-01, discriminator 1.340802e+00
Losses after mini-batch   100: generator 7.938173e-01, discriminator 1.337631e+00
Losses after mini-batch   150: generator 7.152455e-01, discriminator 1.340306e+00
Losses after mini-batch   200: generator 6.683345e-01, discriminator 1.381153e+00
Losses after mini-batch   250: generator 6.697301e-01, discriminator 1.366428e+00
Losses after mini-batch   300: generator 8.746617e-01, discriminator 1.312445e+00
Losses after mini-batch   350: generator 7.787442e-01, discriminator 1.362245e+00
Losses after mini-batch   400: generator 7.511564e-01, discriminator 1.328067e+00
Losses after mini-batch   450: generator 7.802117e-01, discriminator 1.376372e+00


Starting epoch 45 with 469 batches...
Losses after mini-batch    50: generator 7.600512e-01, discriminator 1.383873e+00
Losses after mini-batch   100: generator 8.160909e-01, discriminator 1.358708e+00
Losses after mini-batch   150: generator 7.226414e-01, discriminator 1.354639e+00
Losses after mini-batch   200: generator 8.329083e-01, discriminator 1.312028e+00
Losses after mini-batch   250: generator 7.039732e-01, discriminator 1.362642e+00
Losses after mini-batch   300: generator 7.578528e-01, discriminator 1.323241e+00
Losses after mini-batch   350: generator 7.364562e-01, discriminator 1.362296e+00
Losses after mini-batch   400: generator 7.791233e-01, discriminator 1.302801e+00
Losses after mini-batch   450: generator 9.140904e-01, discriminator 1.364608e+00


Starting epoch 46 with 469 batches...
Losses after mini-batch    50: generator 7.639457e-01, discriminator 1.285058e+00
Losses after mini-batch   100: generator 7.593336e-01, discriminator 1.352478e+00
Losses after mini-batch   150: generator 7.456541e-01, discriminator 1.390308e+00
Losses after mini-batch   200: generator 6.982637e-01, discriminator 1.343951e+00
Losses after mini-batch   250: generator 7.573276e-01, discriminator 1.327403e+00
Losses after mini-batch   300: generator 7.050091e-01, discriminator 1.358659e+00
Losses after mini-batch   350: generator 7.636526e-01, discriminator 1.365343e+00
Losses after mini-batch   400: generator 7.119561e-01, discriminator 1.324384e+00
Losses after mini-batch   450: generator 7.904121e-01, discriminator 1.329813e+00


Starting epoch 47 with 469 batches...
Losses after mini-batch    50: generator 7.064092e-01, discriminator 1.342235e+00
Losses after mini-batch   100: generator 7.888452e-01, discriminator 1.315889e+00
Losses after mini-batch   150: generator 7.771449e-01, discriminator 1.310146e+00
Losses after mini-batch   200: generator 7.131274e-01, discriminator 1.326784e+00
Losses after mini-batch   250: generator 8.066496e-01, discriminator 1.327012e+00
Losses after mini-batch   300: generator 8.047030e-01, discriminator 1.362932e+00
Losses after mini-batch   350: generator 9.941527e-01, discriminator 1.333535e+00
Losses after mini-batch   400: generator 7.116566e-01, discriminator 1.300182e+00
Losses after mini-batch   450: generator 6.952834e-01, discriminator 1.345784e+00


Starting epoch 48 with 469 batches...
